In [1]:
# TODO: move this out of the repo to jupyter repository
import time
import threading
from IPython.display import display
import ipywidgets as widgets


class LeakyBucket:
    def __init__(self, capacity, leak_rate):
        self.capacity = capacity
        self.leak_rate = leak_rate
        self.queue = []
        self.last_leak_time = time.time()
        self.lock = threading.Lock()
        self.size_widget = widgets.IntText(description="Bucket Size:", disabled=True)
        display(self.size_widget)
        self.running = True  # Flag to control the running state

    def add_task(self, task):
        with self.lock:
            if len(self.queue) < self.capacity:
                self.queue.append(task)
            else:
                print("Bucket is full. Task rejected.")
        self.size_widget.value = len(self.queue)

    def leak_tasks(self):
        while self.running:  # Check the running flag
            with self.lock:
                now = time.time()
                leak_count = int((now - self.last_leak_time) * self.leak_rate)

                for _ in range(min(leak_count, len(self.queue))):
                    self.queue.pop(0)()

                self.last_leak_time = now
            self.size_widget.value = len(self.queue)
            time.sleep(1 / self.leak_rate)

    def stop(self):  # Method to stop the threads
        self.running = False


def add_tasks(bucket, rate):
    while bucket.running:  # Check the running flag
        bucket.add_task(my_task)
        time.sleep(1 / rate)


def my_task():
    print("Task executed at:", time.strftime("%H:%M:%S"))


def main():
    bucket = LeakyBucket(capacity=10, leak_rate=3)

    # Start the leaking thread
    leak_thread = threading.Thread(target=bucket.leak_tasks)
    leak_thread.daemon = True
    leak_thread.start()

    # Start the adding thread at twice the leak rate
    add_thread = threading.Thread(target=add_tasks, args=(bucket, 6))
    add_thread.daemon = True
    add_thread.start()

    return bucket  # Return the bucket instance for later access


bucket = main()

IntText(value=0, description='Bucket Size:', disabled=True)

Task executed at: 19:00:23
Task executed at: 19:00:23
Task executed at: 19:00:23
Task executed at: 19:00:24
Task executed at: 19:00:24
Task executed at: 19:00:24
Task executed at: 19:00:25
Task executed at: 19:00:25
Task executed at: 19:00:26
Bucket is full. Task rejected.
Task executed at: 19:00:26
Bucket is full. Task rejected.
Task executed at: 19:00:26
Bucket is full. Task rejected.
Task executed at: 19:00:27
Bucket is full. Task rejected.
Task executed at: 19:00:27
Bucket is full. Task rejected.
Task executed at: 19:00:27
Bucket is full. Task rejected.
Task executed at: 19:00:28
Bucket is full. Task rejected.
Task executed at: 19:00:28
Bucket is full. Task rejected.
Task executed at: 19:00:28
Bucket is full. Task rejected.
Task executed at: 19:00:29
Bucket is full. Task rejected.
Task executed at: 19:00:29
Bucket is full. Task rejected.
Task executed at: 19:00:29
Bucket is full. Task rejected.
Task executed at: 19:00:30
Bucket is full. Task rejected.
Task executed at: 19:00:30
Buc

In [ ]:
# To stop the threads, you can call the stop method of the bucket
# For example, you can do this in a separate cell in the Jupyter Notebook
# bucket.stop()